In [7]:
from neo4j import GraphDatabase
import pandas as pd

uri = "neo4j://localhost:7687"  
user = "neo4j"               
password = "datathink"        

driver = GraphDatabase.driver(uri, auth=(user, password))

In [30]:
# Import site metadata
summary_stats_2022 = pd.read_csv("Neo4JOutputs/count_level_summary.csv")
joining_sites = pd.read_csv("july2023_hsi.csv")
joining_table = joining_sites[["Site Name", "County"]]

In [ ]:
def execute_query_general(query, parameters=None):
    with driver.session() as session:
        result = session.run(query, parameters)
        return [record for record in result]

In [37]:
# Define function that can basically obtain counts 
# We will
def execute_query(query,parameters=None):
    with driver.session() as session:
        result = session.run(query, parameters)
        return [{"Site Name": record["Site Name"], "counts": record["counts"], "latitude":record["latitude"], "longitude":record["longitude"]} for record in result]

def query_constructor(node_type, pollutant_area):
    query = "MATCH (env:Node {id: " + "'" + node_type + "'})-[edge:RELATES_TO]-(pointed:Node)-[tosite:RELATES_TO {category: "+ "'" + pollutant_area + "'}]-(sitenode:Node {node_class: 'Site'}) \
    RETURN sitenode.label AS `Site Name`, sitenode.latitude AS latitude, sitenode.longitude AS longitude, count(*) AS counts \
    ORDER BY counts DESC"

    return query

results = execute_query(query_constructor("Carcinogenicity", "Water"))

df = pd.DataFrame(results)

df


,Site Name,counts,latitude,longitude
0,TH Agriculture & Nutrition Company - NPL Site,5,31.61722,-84.18167
1,M&J Solvents Company,5,33.795,-84.43806
2,Atlanta Gas Light Company - Waycross MGP Site,5,31.19889,-82.37389
3,Robins Air Force Base - NPL Site,5,32.62083,-83.58167
4,"Amrep, Inc. - Marietta",5,33.97583,-84.53944
...,...,...,...,...
171,Southside Cleaners,1,31.97361,-81.13306
172,Cork's Fabricare,1,31.22333,-81.47833
173,Lowe's Home Improvement Warehouse,1,33.55083,-84.41389
174,Chase Road Property,1,34.51778,-83.51194


In [16]:
# Establish function to dynamically return available nodes in the db
def get_nodes_by_class(node_class):
    query = "MATCH (n) WHERE n.node_class = $node_class RETURN n.id"
    print(query)
    with driver.session() as session:
        result = session.run(query, node_class=node_class)
        return [record["n.id"] for record in result]

node_class = "Chem Desc"
node_names = get_nodes_by_class(node_class)

MATCH (n) WHERE n.node_class = $node_class RETURN n.id
['Cardiotoxicity', 'IGC50', 'Carcinogenicity', 'LC50', 'Hepatotoxicity', 'Dev_Toxic', 'Respiratory', 'Repo_Toxic', 'BCF', 'Endocrine_NR-AR', 'LCM50DM', 'Mutagenicity']


In [17]:
node_names

['Cardiotoxicity',
 'IGC50',
 'Carcinogenicity',
 'LC50',
 'Hepatotoxicity',
 'Dev_Toxic',
 'Respiratory',
 'Repo_Toxic',
 'BCF',
 'Endocrine_NR-AR',
 'LCM50DM',
 'Mutagenicity']

# Prepare data for shiny app

In [108]:
water_data = {}
soil_data = {}
for node in node_names:
    results = execute_query(query_constructor(node, "Water"))

    df = pd.DataFrame(results)
    df.to_csv("./water/" + node + ".csv")

    water_data.update({node:df})

for node in node_names:
    results = execute_query(query_constructor(node, "Soil"))

    df = pd.DataFrame(results)
    df.to_csv("./soil/" + node + ".csv")

    soil_data.update({node:df})


In [39]:
soil_data

{'Cardiotoxicity': Empty DataFrame
 Columns: []
 Index: [],
 'IGC50':                              Site Name  counts  latitude  longitude
 0     Robins Air Force Base - NPL Site       8  32.62083  -83.58167
 1    Atlantic Wood Industries (Former)       7  32.14583  -81.14917
 2       Seaboard Industrial Blvd. Site       7  33.80833  -84.43167
 3    Paper Mill Road Undeveloped Tract       6   33.9525  -83.96278
 4     Pilot Wastewater Treatment Plant       5  32.97278  -85.02667
 ..                                 ...     ...       ...        ...
 113          Polymer Specialties, Inc.       1  34.98111    -85.435
 114             Amrep, Inc. - Marietta       1  33.97583  -84.53944
 115                Chase Road Property       1  34.51778  -83.51194
 116          Square D Company (Former)       1  33.79278     -84.43
 117                 Thomas & Co., Inc.       1  33.26667  -84.25889
 
 [118 rows x 4 columns],
 'Carcinogenicity':                                              Site Name  

In [72]:
summary_stats_2022[["Geography", "avg_pop", "percent_black", "poverty_rate", "age_18_24_score", "educational_score_25_over"]]

,Geography,avg_pop,percent_black,poverty_rate,age_18_24_score,educational_score_25_over
0,Appling,18136.826087,19.763404,23.422823,1.052459,1.312125
1,Atkinson,8189.739130,15.629888,28.172589,0.983951,1.200265
2,Bacon,10906.217391,16.445242,26.110090,1.152807,1.342518
3,Baker,3462.565217,38.220065,20.614887,1.095238,1.630890
4,Baldwin,45536.956522,42.048278,23.720588,1.548893,1.740812
...,...,...,...,...,...,...
154,Whitfield,99070.782609,3.462285,16.242597,1.357603,1.345530
155,Wilcox,8870.434783,35.570624,25.026257,0.991848,1.420438
156,Wilkes,10222.173913,41.890375,22.440087,1.189459,1.333808
157,Wilkinson,9485.739130,39.038569,24.651004,1.282454,1.376569


In [80]:
import json
import plotly.graph_objs as go

with open('Neo4JOutputs/geojson-counties-fips.json') as file:
    counties_geojson = json.load(file)

def plot_feature(pulldata, site_var, county_var, location):
    georgia_features = [feature for feature in counties_geojson['features'] if feature['properties']['STATE'] == '13']

    georgia_geojson = {
        "type": "FeatureCollection",
        "features": georgia_features
    }

    chemical_colorscale = [
        [0, 'rgb(0, 50, 50)'],
        [0.5, 'rgb(0, 100, 50)'],
        [1, 'rgb(0, 0, 200)']
    ]

    # Create a scatter plot for the sites
    scatter = go.Scattergeo(
        lon = pulldata['longitude'],
        lat = pulldata['latitude'],
        text = pulldata['Site Name'],
        mode = 'markers',
        marker = dict(
            color = pulldata['counts'],
            colorscale = chemical_colorscale,
            colorbar = dict(
                title = 'Number of Chemicals',
                x = 0.1
            ),
            opacity = 0.5,
            reversescale = False,
            symbol = 'circle',
            showscale = True
        )
    )

    # Create a map with both the scatter plot and the county boundaries
    fig = go.Figure(scatter)
    fig.update_geos(
        fitbounds="locations",
        visible=False
    )
    fig.add_trace(
        go.Choropleth(
            geojson=georgia_geojson,
            featureidkey="properties.NAME",
            locations=summary_stats_2022['Geography'],
            z=summary_stats_2022[county_var],
            colorscale="Reds",
            colorbar = dict(
                title = county_var,
                x = 0.9
            ),
            marker_line_width=0
        )
    )
    fig.update_layout(
        title_text = county_var + ' & ' + site_var + " - " + location,
        geo_scope='usa',
    )

    # Show the figure
    return fig

,County,counts,population_category,black_percentile_category,poor_percentile_category,age_18_24_percentile_category,educational_score_25_over_percentile_category
9,Chatham,54,very high,high,low,very high,very high
24,Fulton,33,very high,very high,low,very high,very high
13,Cobb,16,very high,medium,very low,very high,very high
26,Gwinnett,15,very high,medium,very low,very high,very high
41,Richmond,13,very high,very high,high,high,high
20,Dougherty,13,very high,very high,very high,high,high
25,Glynn,12,very high,medium,medium,medium,very high
50,Troup,12,high,high,medium,high,high
48,Tift,12,high,medium,high,very high,medium
43,Spalding,11,high,high,medium,medium,medium


In [90]:
summary_stats_2022

,Unnamed: 0,Geography,avg_pop,population_category,percent_black,black_percentile_category,poverty_rate,poor_percentile_category,age_18_24_score,educational_score_25_over,age_18_24_percentile_category,educational_score_25_over_percentile_category,I,IV,II,V,total_sites,SitesCategory,group_identifier,years
0,1,Appling,18136.826087,medium,19.763404,low,23.422823,high,1.052459,1.312125,very low,very low,1,0,0,0,1,>=1 Site,medium.low.high.very low,1994
1,2,Atkinson,8189.739130,very low,15.629888,low,28.172589,very high,0.983951,1.200265,very low,very low,1,0,0,0,1,>=1 Site,very low.low.very high.very low,1994
2,3,Bacon,10906.217391,low,16.445242,low,26.110090,very high,1.152807,1.342518,low,very low,0,1,0,0,1,>=1 Site,low.low.very high.very low,2002
3,4,Baker,3462.565217,very low,38.220065,high,20.614887,medium,1.095238,1.630890,low,high,0,0,0,0,0,0 Site,very low.high.medium.high,NaN
4,5,Baldwin,45536.956522,high,42.048278,very high,23.720588,high,1.548893,1.740812,very high,high,0,1,0,0,1,>=1 Site,high.very high.high.high,1995
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
154,155,Whitfield,99070.782609,very high,3.462285,very low,16.242597,low,1.357603,1.345530,high,low,0,2,0,0,2,>=1 Site,very high.very low.low.low,"1997, 2001"
155,156,Wilcox,8870.434783,very low,35.570624,high,25.026257,very high,0.991848,1.420438,very low,low,0,0,0,0,0,0 Site,very low.high.very high.low,NaN
156,157,Wilkes,10222.173913,low,41.890375,high,22.440087,high,1.189459,1.333808,low,very low,0,1,0,0,1,>=1 Site,low.high.high.very low,1997
157,158,Wilkinson,9485.739130,very low,39.038569,high,24.651004,very high,1.282454,1.376569,medium,low,0,1,0,0,1,>=1 Site,very low.high.very high.low,2001


In [105]:
from shiny import App, ui, reactive, render, Inputs, Outputs, Session
import nest_asyncio
import pandas as pd
from plotly.io import to_html
import plotly.express as px

nest_asyncio.apply()

# Assuming soil_data and water_data are defined somewhere in your script

app_ui = ui.page_fluid(
    ui.navset_tab(
        # Tab 1: View Data
        ui.nav("View Data", "View Data", ui.page_fluid(
            ui.input_select("select_type", "Select Type:", ["Soil", "Water"]),
            ui.input_select("select_key", "Select Key:", list(water_data.keys())),
            ui.output_ui("data_display"),
        )),
        # Tab 2: View Agg Data
        ui.nav("View County Stats", "View County Stats", ui.page_fluid(
            ui.input_select("select_type_county", "Select Type:", ["Soil", "Water"]),
            ui.input_select("select_key_county", "Select Key:", list(water_data.keys())),
            ui.input_select("filter_bool", "Filter?", ["Yes", "No"]),
            ui.input_select("filterby", "Filter By:", ["population_category", "black_percentile_category", "poor_percentile_category", "age_18_24_percentile_category", "educational_score_25_over_percentile_category"]),
            ui.input_select("filterbylevel", "Filter Level:", ["very low", "low", "medium", "high", "very high"]),
            ui.output_ui("data_display_agg"),
        )),
        # Tab 3: Map View
        ui.nav("Map View", "Map View", ui.div(
            ui.input_select("select_type_map", "Select Type:", ["Soil", "Water"]),
            ui.input_select("select_key_map", "Select Key:", list(water_data.keys())),
            ui.input_select("select_countvar_map", "Select County Variable", ["avg_pop", "percent_black", "poverty_rate", "age_18_24_score", "educational_score_25_over"]),
            ui.output_ui("map_display")
        )),
        # Tab 4: Statistics (Eventually)
        ui.nav("Plot County Trends", "Plot County Trends", ui.div(
            ui.input_select("select_type_trends", "Select Type:", ["Soil", "Water"]),
            ui.input_select("select_key_trends", "Select Key:", list(water_data.keys())),
            ui.input_select("plot_county", "Plot County:", ["avg_pop", "percent_black", "poverty_rate", "age_18_24_score", "educational_score_25_over"]),
            ui.output_ui("stats_display")
        ))
    )
)

# Server logic
def server(input, output, session: Session):

    @output
    @render.ui
    def data_display():
        selected_type = input.select_type()
        selected_key = input.select_key()
        data = soil_data[selected_key] if selected_type == "Soil" else water_data[selected_key]
        return ui.HTML(data.to_html(classes="table table-striped"))

    @output
    @render.ui
    def data_display_agg():
        selected_type = input.select_type_county()
        selected_key = input.select_key_county()
        data = soil_data[selected_key] if selected_type == "Soil" else water_data[selected_key]

        merge1 = pd.merge(left = data, right = joining_table)[["County", "counts"]]

        merge1_agg = merge1.groupby("County").agg("sum").reset_index()

        return_data = pd.merge(merge1_agg, summary_stats_2022[["Geography","population_category", "black_percentile_category", "poor_percentile_category", "age_18_24_percentile_category", "educational_score_25_over_percentile_category"]], left_on = "County", right_on = "Geography").drop(["Geography"], axis = 1).sort_values(by = "counts", ascending= False)

        if input.filter_bool() == "Yes":
            selected_col = input.filterby()
            level = input.filterbylevel()

            return_data = return_data.loc[return_data[selected_col] == level ,:]
                
        return ui.HTML(return_data.to_html(classes="table table-striped"))

    @output
    @render.ui
    def map_display():
        selected_type = input.select_type_map()
        selected_key = input.select_key_map()
        selected_county_var = input.select_countvar_map()
        data = soil_data[selected_key] if selected_type == "Soil" else water_data[selected_key]

        fig = plot_feature(data, selected_key, selected_county_var, selected_type)

        plot_html = to_html(fig, full_html=False, include_plotlyjs='cdn')
        return ui.HTML(plot_html)

    @output
    @render.ui
    def stats_display():
        selected_type = input.select_type_trends()
        selected_key = input.select_key_trends()
        data = soil_data[selected_key] if selected_type == "Soil" else water_data[selected_key]

        merge1 = pd.merge(left = data, right = joining_table)[["County", "counts"]]

        merge1_agg = merge1.groupby("County").agg("sum").reset_index()

        return_data = pd.merge(merge1_agg, summary_stats_2022[["Geography","avg_pop", "percent_black", "poverty_rate", "age_18_24_score", "educational_score_25_over"]], left_on = "County", right_on = "Geography").drop(["Geography"], axis = 1).sort_values(by = "counts", ascending= False)

        plotthis = input.plot_county()

        fig = px.scatter(return_data, x = plotthis, y = "counts", hover_data = ["County"])

        plot_html = to_html(fig, full_html=False, include_plotlyjs='cdn')

        return ui.HTML(plot_html)

app = App(app_ui, server)
app.run()

INFO:     Started server process [91382]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:57686 - "GET / HTTP/1.1" 200 OK


INFO:     ('127.0.0.1', 57688) - "WebSocket /websocket/" [accepted]
INFO:     connection open
Traceback (most recent call last):
  File "/Users/kenminsoo/opt/anaconda3/envs/datathink/lib/python3.9/site-packages/shiny/session/_session.py", line 1023, in output_obs
    message[output_name] = renderer_fn()
  File "/Users/kenminsoo/opt/anaconda3/envs/datathink/lib/python3.9/site-packages/shiny/render/transformer/_transformer.py", line 407, in __call__
    return run_coro_sync(self._run())
  File "/Users/kenminsoo/opt/anaconda3/envs/datathink/lib/python3.9/site-packages/shiny/_utils.py", line 304, in run_coro_sync
    coro.send(None)
  File "/Users/kenminsoo/opt/anaconda3/envs/datathink/lib/python3.9/site-packages/shiny/render/transformer/_transformer.py", line 322, in _run
    ret = await self._transformer(
  File "/Users/kenminsoo/opt/anaconda3/envs/datathink/lib/python3.9/site-packages/shiny/render/_render.py", line 523, in UiTransformer
    ui = await resolve_value_fn(_fn)
  File "/User